# Outline

## 1. Packages
- Packages pulled from various weeks for classifiers
### 1.1 Decision Trees
### 1.2 Linear Regression
### 1.3 Neural Nets
### 1.4 SVM

## 2. Import Data
- Currently only 2016 data
### 2.1 Assign parcelid to dataframe indicies and subset data
Contains three lines of code that can only be run once

## 3. Check the data
- Describe the data and check that everything makes sense
### 3.1 Data shape
### 3.2 Descriptive stats of train data
### 3.3 Variables types

## 4. Initial data prep
- First run of data prep, includes subsetting columns, shuffling and a first run training set
### 4.1 Test Subsetting (subsetting columns of the data)
### 4.2 Shuffle Data
### 4.4 First run testing data

## 5. Linear Regression
- Fit a linear regression
- Issues so far: 
    - not all data was int/float (see 4.1 for subsetting out non-int and float objects)
    - NaN values

# Considerations
- Should we add the transaction date from the train labels to the train data?

In [3]:
"""
1. Packages
All the packages we'll need for the various algorithms we can use
"""

# Global imports
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline

np.random.seed(0)

"""
1.1 Decision Trees
Source: Week 4 Notebook


"""

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# For producing decision tree diagrams.
from IPython.core.display import Image, display
from sklearn.externals.six import StringIO
import pydot

"""
1.2 Linear Regression
Source: Week 6 Notebook


"""

import seaborn as sns
from numpy.linalg import inv
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

np.set_printoptions(precision=4, suppress=True)

"""
1.3 Neural Network
Source: Week 7 Notebook and Week 10 notebook

- Jason recommends we use Keras, but I have included the Week 7 imports as well


from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print(theano.config.device) # We're using CPUs (for now)
print(theano.config.floatX) # Should be 64 bit for CPUs

"""

import keras
keras.__version__

"""
1.4 SVM
Source: Week 10 Notebook

from sklearn.linear_model import LinearRegression
"""

from scipy import stats
import pylab as pl

sns.set()

Using TensorFlow backend.


In [5]:
"""
2. Import data
Import the Zillow Home Value Prediction data from the Kaggle Competition

Try to run this a few times as possible, the data set takes a little while to import
"""

# Import the data dictionary for a summary of the features we have
data_dictionary = pd.read_csv('..\Data\zillow_data_dictionary\zillow_data_dictionary.csv', sep=',')
print("Data dictionary done")

# Import the 2016 data
data_2016 = pd.read_csv('..\Data\properties_2016\properties_2016.csv', sep=',', engine='python')
print("Data 2016 done")

# Import 2016 labels
train_2016 = pd.read_csv('../Data/train_2016_v2/train_2016_v2.csv', sep=",", engine="python")
print("Train 2016 done")

# Check the shape
print('data shape:', data_2016.shape)

Data dictionary done
Data 2016 done
Train 2016 done
data shape: (2985217, 58)


In [21]:
"""
2.1 Assign parcelid to dataframe indicies and subset data
"""

# Only run these once
"""
data_2016 = data_2016.set_index('parcelid') # can only run once, function removes 'parcelid' from the df
train_data_2016 = data_2016.loc[train_2016.parcelid] # also only run once to avoid unnecesary processing
train_2016 = train_2016.set_index('parcelid') # also only run once
"""

print(data_2016.airconditioningtypeid.head(5))
print(train_2016.head(5))
print(train_data_2016.airconditioningtypeid.head(5))

parcelid
10754147   NaN
10759547   NaN
10843547   NaN
10859147   NaN
10879947   NaN
Name: airconditioningtypeid, dtype: float64
          logerror transactiondate
parcelid                          
11016594    0.0276      2016-01-01
14366692   -0.1684      2016-01-01
12098116   -0.0040      2016-01-01
12643413    0.0218      2016-01-02
14432541   -0.0050      2016-01-02
parcelid
11016594    1.0
14366692    NaN
12098116    1.0
12643413    1.0
14432541    NaN
Name: airconditioningtypeid, dtype: float64


In [13]:
# 3. Check the data

# 3.1 Data shape
# Check the shape of the train data
print("Train data shape:", train_2016.shape, "Subsetted train data shape:", train_data_2016.shape)

# 3.2 Descriptive stats of train data
# Descriptive statistics of train data
train_data_2016.describe()

Train data shape: (90275, 3) Subsetted train data shape: (90275, 57)


,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,28781.000000,261.000000,43.000000,90275.000000,90275.000000,16.0,57364.000000,89093.000000,658.0,6856.000000,...,95.000000,89519.000000,20570.000000,8.989500e+04,9.027400e+04,90275.0,9.027400e+04,90269.000000,1783.000000,8.967000e+04
mean,1.816372,7.229885,713.581395,2.279474,3.031869,4.0,5.565407,2.309216,66.0,1347.974037,...,311.694737,1968.532870,1.440739,1.800934e+05,4.576726e+05,2015.0,2.783353e+05,5983.975927,13.402692,6.049151e+13
std,2.974168,2.716196,437.434198,1.004271,1.156436,0.0,1.900602,0.976172,0.0,652.399026,...,346.354850,23.763475,0.544498,2.091299e+05,5.548844e+05,0.0,4.004955e+05,6838.876956,2.715966,2.046605e+11
min,1.000000,2.000000,100.000000,0.000000,0.000000,4.0,1.000000,1.000000,66.0,44.000000,...,18.000000,1885.000000,1.000000,1.000000e+02,2.200000e+01,2015.0,2.200000e+01,49.080000,6.000000,6.037101e+13
25%,1.000000,7.000000,407.500000,2.000000,2.000000,4.0,4.000000,2.000000,66.0,938.000000,...,100.000000,1953.000000,1.000000,8.124500e+04,1.990232e+05,2015.0,8.222800e+04,2872.830000,13.000000,6.037320e+13
50%,1.000000,7.000000,616.000000,2.000000,3.000000,4.0,7.000000,2.000000,66.0,1244.000000,...,159.000000,1970.000000,1.000000,1.320000e+05,3.428720e+05,2015.0,1.929700e+05,4542.750000,14.000000,6.037615e+13
75%,1.000000,7.000000,872.000000,3.000000,4.000000,4.0,7.000000,3.000000,66.0,1614.000000,...,361.000000,1987.000000,2.000000,2.105345e+05,5.405890e+05,2015.0,3.454195e+05,6901.090000,15.000000,6.059042e+13
max,13.000000,21.000000,1555.000000,20.000000,16.000000,4.0,12.000000,20.000000,66.0,7625.000000,...,1366.000000,2015.000000,4.000000,9.948100e+06,2.775000e+07,2015.0,2.450000e+07,321936.090000,99.000000,6.111009e+13


In [35]:
# 3.3 Variables types
# Print the type of each variable

print("ALL DATA TYPES")
print(train_data_2016.dtypes)

ALL DATA TYPES
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                   object
heatingorsystemtypeid           float64
latitude                        float64
longitude                

In [36]:
# 4. Prep the data

# 4.1 Test Subsetting
# Testing - keep only int or float columns, to just initially test algorithms before doing any additional data manipulation
temp_data = train_data_2016.select_dtypes(['float64','int64'])

# Check Variable types
print("\nINT AND FLOAT ONLY DATA")
print(temp_data.dtypes)

In [ ]:
# 4.2 Shuffle Data
# Shuffle the data
# Shuffling the subsetted 2016 training data
np_data_2016, np_train_2016 = np.asarray(temp_data), np.asarray(train_2016['logerror'])
shuffle = np.random.permutation(np.arange(np_data_2016.shape[0]))
np_data_2016, np_train_2016 = np_data_2016[shuffle], np_train_2016[shuffle]

In [42]:
# 4.4 First run testing data
mini_train_data_2016, mini_train_labels_2016 = np_data_2016[:10000], np_train_2016[:10000]

In [43]:
# 5. Linear Regression
linr =  LinearRegression()
linr.fit(mini_train_data_2016, mini_train_labels_2016)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').